# "Eleições RJ: Votação dos Deputados por Bairros (capital)"
> "dados do TSE"

- toc: true
- branch: master
- badges: true
- comments: false
- numbersections: true
- categories: [eleições, RJ, bairros]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true

In [1]:
#hide_input

import os
from IPython.core.display import display, HTML
from IPython.display import IFrame
# import matplotlib.pyplot as plt
import pandas as pd
import json

import electionsBR as el
import LabFunctions as lab
from EppDataBase import EppDataBase

def get_dict_candidatxs(uf,ano,cargo,partidos,only_elected,epp,nr_partidos=[]):
    if cargo == 'VEREADOR':
        # dict_candidatxs = {}
        # only_elected = False
        df = epp.get_results_df(ano,uf)
        df_dict = df.loc[df['nr_partido'].isin(nr_partidos)].copy()
        df_dict['NR_VOTAVEL'] = df_dict['NR_VOTAVEL'].astype(str)
        df_dict['nome'] = df_dict['NR_VOTAVEL'] + '_' + df_dict['NM_VOTAVEL']
        dict_candidatxs = df_dict.set_index('NR_VOTAVEL')['nome'].to_dict()
        
    else:
        elected = get_elected_candidates_dict(uf,partidos,cargos=[cargo],anos=[ano],only_elected=only_elected)
        df = elected[cargo][ano].copy()
        df['num_nome'] = df['NUMERO_CANDIDATO'] + '_' + df['NOME_CANDIDATO']
        dict_candidatxs = df.set_index('NUMERO_CANDIDATO')['num_nome'].to_dict()
        
        dict_candidatxs['13'] = '13_PT'
        dict_candidatxs['50'] = '50_PSOL'
        dict_candidatxs['65'] = '65_PCdoB'
        
    return dict_candidatxs

def show_folium(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    # display(HTML(data_fixed_height))
    return data_fixed_height

def get_df_style(df_bairros,epp,ano,uf,n_turno,cargo,municipio,partidos,only_elected,group_level,selected_col,relative, nr_partidos,nrows=None,columns_dict=None):
    selection_dict = {'NR_TURNO':n_turno,'DS_CARGO':cargo,'NM_MUNICIPIO':municipio}
    # dict_candidatxs = get_dict_candidatxs(elected,ano,cargo)
    # partidos=partidos_esquerda

    # if cargo == 'VEREADOR':
    #     # dict_candidatxs = {}
        
    #     df = epp.get_results_df(ano,uf)
    #     df_dict = df.loc[df['nr_partido'].isin(nr_partidos)].copy()
    #     df_dict['NR_VOTAVEL'] = df_dict['NR_VOTAVEL'].astype(str)
    #     df_dict['nome'] = df_dict['NR_VOTAVEL'] + ' - ' + df_dict['NM_VOTAVEL']
    #     dict_candidatxs = df_dict.set_index('NR_VOTAVEL')['nome'].to_dict()
    #     only_elected = False
    # else:
    #     dict_candidatxs = get_dict_candidatxs(uf,ano,cargo,partidos=partidos,only_elected=only_elected)
    #     dict_candidatxs['13'] = '13 - PT'
    #     dict_candidatxs['50'] = '50 - PSOL'
    #     dict_candidatxs['65'] = '65 - PCdoB'

    # df_bairros = lab.get_group_results(ano,uf,selection_dict,group_level,selected_col=selected_col,relative=relative,dict_candidatxs=dict_candidatxs)

    # selected_cols = [col for col in df_bairros.columns if col[:2] in numeros_partidos]
    # df_bairros = df_bairros[selected_cols]


    sufix = 'eleitos_' if only_elected else 'todos_'
    sufix += 'relativo' if relative else 'absoluto'
    folder_path = 'results/votacao_NM_BAIRRO'
    os.makedirs(folder_path,exist_ok=True)
    fname = f'{ano}_RJ_{cargo}_{"_".join(partidos)}_{sufix}.csv'.replace(' ','_')
    df_bairros = df_bairros.reset_index(drop=True)
    if not relative:
        df_bairros.iloc[:,2:] = df_bairros.iloc[:,2:].astype(int)
        # df_bairros.iloc[:,2:] = df_bairros.select_dtypes(include='number').astype(int)
    df_bairros = df_bairros.copy()
    df_bairros['SOMA'] = df_bairros.sum(axis=1,numeric_only=True)
    df_bairros = df_bairros.sort_values(by='SOMA',ascending=False)
    df_bairros.to_csv(f'{folder_path}/{fname}',index=False,sep=';')
    if nrows:
        df_bairros = df_bairros.head(nrows)
    df_bairros.index = range(1,len(df_bairros)+1)

    if columns_dict:
        df_bairros = df_bairros.rename(columns=columns_dict)

    # remove coluna do município
    df_bairros = df_bairros.iloc[:,1:]

    # df_bairros = df_bairros.reset_index()
    # df_bairros = df_bairros.set_index(['index','NM_BAIRRO'])
    font_size = 12
    line_height = 12
    
    hover_color = '#00BBBB'
    hover_color = '#EEEE55'
    styles = []
    # styles = [dict(selector="tr:hover", props=[("background-color", f"{hover_color}")])]
    # styles = [dict(selector="tr:hover", props=[("font-weight", "bold")])]
    # styles.append(dict(selector="", props=[("font-size", "8px")]))
    # styles.append(dict(selector="", props=[("line-height", "6px")]))
    # styles.append(dict(selector="", props=[("text-align", "center")]))
    

    headers = {
        'selector': 'th',
        'props': f'background-color: #cccccc; font-size: {font_size}px; text-align: center'
        # 'props': 'background-color: #000066; color: white;'        
        }

    col0 = {
        'selector': '.col0',
        'props': f'widht:200px; font-size:{font_size-2}px;'
        # 'line-height':f'{line_height}px',
        # 'props': 'background-color: #000066; color: white;'        
        }
    
    tr = {
        'selector': 'tr:hover',
        'props': f'background-color:{hover_color}'
        # 'line-height':f'{line_height}px',
        # 'props': 'background-color: #000066; color: white;'        
        }

    # column_width = 100
    # columns = {
    #     'selector': 'td',
    #     'props': f'width: {column_width}px;'
    #     # 'props': 'background-color: #000066; color: white;'        
    #     }

    styles += [headers,col0,tr]
    format_dict = {}
    for col,tipo in df_bairros.dtypes.items():
        if tipo == 'float64':
            format_dict[col] = '{:,.2%}'.format

    df_style = df_bairros.style.format(format_dict)

    # df_style = df_style

    df_style = df_style.set_properties(**{
                'padding': '0px',
                'font-size':f'{font_size}px',
                'line-height':f'{line_height}px',
                'width': '100px',
                "text-align": "center"})

    df_style = df_style.background_gradient() #.bar(color='#d65f5f')
    df_style= df_style.set_table_styles(styles)

    # df_style = df_style.format('{:,.2f}'.format)
#     'var1': '{:,.2f}'.format,
#     'var2': '{:,.2f}'.format,
#     'var3': '{:,.2%}'.format,
# })

    return df_style

def get_elected_candidates_dict(uf,partidos,cargos,anos,only_elected=True,verbose=False):
    if only_elected:
        folder_path = 'results/eleitos'
    else:
        folder_path = 'results/candidatos'
    os.makedirs(folder_path,exist_ok=True)
    elected = {}
    for cargo in cargos:
        cargo_api = ' '.join([x.capitalize() for x in cargo.split()])
        elected[cargo] = {}
        for ano in anos:
            fname = f'{ano}_{uf}_{cargo}_{"_".join(partidos)}.csv'.replace(' ','_')
            file_path = f'{folder_path}/{fname}'
            if verbose:
                print(file_path)
            if not os.path.exists(file_path):
                if only_elected:
                    cand = el.get_candidates(year=ano, position=cargo_api,only_elected=only_elected,
                        columns=["NUMERO_CANDIDATO", "NOME_CANDIDATO", "SIGLA_UF", "NUM_TURNO", "SIGLA_PARTIDO", "COMPOSICAO_LEGENDA"])
                else:
                    cand = el.get_candidates(year=ano, position=cargo_api,
                        columns=["NUMERO_CANDIDATO", "NOME_CANDIDATO", "SIGLA_UF", "NUM_TURNO", "SIGLA_PARTIDO", "COMPOSICAO_LEGENDA"])

                cand = cand.query(f"SIGLA_UF == '{uf}' ").query(f"SIGLA_PARTIDO in {partidos}")
                cand.to_csv(file_path,index=False)
            else:
                cand = pd.read_csv(file_path)
                cand['NUMERO_CANDIDATO'] = cand['NUMERO_CANDIDATO'].astype(str)
            
            elected[cargo][ano] = cand
            
    return elected

def shrink_html(fname,base_path):
    # geo_json_file='../assets/html/maps/geo_json_coords.txt'
    with open(fname,'r') as f:
        lines = f.readlines()

    out_str = ''
    script_file = fname.replace('.html','.js')
    script_scr = script_file.replace('../',base_path)
    for i,line in enumerate(lines):
        if '</body>' in line:
            out_str += line + f'<script src="{script_scr}"></script>\n'
            script_index = i + 2
            break
        else:
            # print(line)
            out_str += line

    with open(fname,'w') as f:
        f.write(out_str)

    # print(len(out_str),out_str)

    script_str = ''
    for line in lines[script_index:]:
        if '</script>' in line:
            break
        script_str+= line
        # print(line)
        
#     with open(script_file,'w') as f:
#         f.write(script_str)

#     # print(script_file)
#     # print(script_str)
#     # for line in lines[script_index:]:

# def shrink_js(fname):

    geo_json_file='../assets/html/geo_json_coords.json'

    # with open(fname,'r') as f:
    #     lines = f.readlines()
    
    lines = script_str.split('\n')

    out_str = ''
    for i,line in enumerate(lines):
        if '_add' in line:
            if len(line) > 1000:
                index = line.index('{"crs"')
                # print('len(line):',len(line))
                if not os.path.exists(geo_json_file):
                    geo_json_coords = line[index:].strip().rstrip(');')
                    with open(geo_json_file,'w') as f:
                        f.write(geo_json_coords)
                # line = line[:index] + ');'
                line = f"""
        function load_geo_json(file_path) {{
            fetch(file_path)
            .then(response => response.text())
            .then(data => {{
              const obj = JSON.parse(data);
              {line[:index].strip()}obj);
            }})
          }}
            const file_path ='{geo_json_file.replace('../',base_path)}';
            load_geo_json(file_path);
            """
        out_str += line + '\n'

    fname = fname.replace('.html','.js')
    with open(fname,'w') as f:
        f.write(out_str)
    # print(len(out_str))
    # print(fname)



    # for i,line in enumerate(lines):
    #     if '_add' in line:
    #         if len(line) > 1000:
    #             index = line.index('{"crs"')
    #             # print('len(line):',len(line))
    #             if not os.path.exists(geo_json_file):
    #                 geo_json_coords = line[index:]
    #                 with open(geo_json_file,'w') as f:
    #                     f.write(geo_json_coords)
    #             line = line[:index] + '###geo_json###'
    #     out_str += line

    # with open(fname,'w') as f:
    #     f.write(out_str)

# df_style = get_df_style(df_bairros,epp,ano,uf,n_turno,cargo,municipio,partidos,only_elected,group_level,selected_col,relative, nr_partidos,nrows,inv_dict)
# df_style

In [2]:
#hide

# definições gerais

n_turno = 1
uf = 'RJ'
municipio= 'RIO DE JANEIRO'
group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
print(f'município selecionado: {municipio}')
print(f'group_level: {group_level}')
        
anos = [2018]
# cargos= ["DEPUTADO FEDERAL","DEPUTADO ESTADUAL"]
cargos= ["Deputado Federal","Deputado Estadual"]
# cargos= ["Deputado Federal"]

only_elected=True
partidos = ['PT','PSOL','PC do B'] # partidos_esquerda
nr_partidos=[13,50,65]

selected_col='NR_VOTAVEL'
nrows=50
        
# cand_elections_fed = {}
epp = EppDataBase()

base_path = '/epp/'
abs_folder = f'{base_path}assets/html/'
relative_folder = '../assets/html'

tables_folder = f'{relative_folder}/tables'
os.makedirs(tables_folder,exist_ok=True)
bairros_geo = 'geojson/limites_bairros_RJ_edit.geojson'

maps_folder = f'{relative_folder}/maps'
os.makedirs(maps_folder,exist_ok=True)

update_maps=True

html_dict = {}


for cargo in cargos:
    html_dict[cargo] = {}
    selection_dict = {'NR_TURNO':n_turno,'NM_MUNICIPIO':municipio,'DS_CARGO':cargo}
    for ano in anos:
        html_dict[cargo][ano]={}
        for relative in [False,True]:
            rel_sufix = 'relativo' if relative else 'absoluto'
            print(f'cargo: {cargo} - ano: {ano} - {rel_sufix}')
            html_dict[cargo][ano][rel_sufix] = {}
            table_name = f'{uf}_{municipio}_{cargo}_{ano}_{rel_sufix}'.replace(' ','_')
            table_path = f'{tables_folder}/{table_name}.html'
            html_dict[cargo][ano][rel_sufix]['tabela'] = f'{abs_folder}/tables/{table_name}.html'
            dict_candidatxs = get_dict_candidatxs(uf,ano,cargo,partidos,only_elected,epp,nr_partidos)
            df_bairros = lab.get_group_results(ano,uf,selection_dict,group_level,selected_col=selected_col,relative=relative,dict_candidatxs=dict_candidatxs)
            
            inv_dict = {v: k for k, v in dict_candidatxs.items()}

            df_style = get_df_style(df_bairros,epp,ano,uf,n_turno,cargo,municipio,partidos,only_elected,group_level,selected_col,relative, nr_partidos,nrows,inv_dict)
            df_style.to_html(table_path)
            
            html_dict[cargo][ano][rel_sufix]['mapas'] = {}
            for num,nome_cand in dict_candidatxs.items():
                map_name = f'{table_name}_{num}'
                html_dict[cargo][ano][rel_sufix]['mapas'][nome_cand] = f'{abs_folder}/maps/{map_name}.html'
                file_path = f'{maps_folder}/{map_name}.html'
                if update_maps:
                    m = lab.get_map(bairros_geo,df_bairros,nome_cand,0.0)
                    m.save(file_path)
                    shrink_html(file_path,base_path)

with open(f'{relative_folder}/html_dict.json', 'w') as fp:
    json.dump(html_dict, fp,sort_keys=True, indent=4)
# with open(f'{maps_folder}/maps_dict.json', 'w') as fp:
#     json.dump(maps_dict, fp,sort_keys=True, indent=4)
# tables_dict
# maps_dict
# html_dict

município selecionado: RIO DE JANEIRO
group_level: NM_BAIRRO
cargo: Deputado Federal - ano: 2018 - absoluto
Adicionando resultado...
Lendo dados do arquivo 'data/votacao_secao_2018_RJ/votacao_secao_2018_RJ.csv'.
Adicionando locais de votação...
Lendo dados do arquivo 'data/eleitorado_local_votacao_2018/eleitorado_local_votacao_2018.csv'.
cargo: Deputado Federal - ano: 2018 - relativo
cargo: Deputado Estadual - ano: 2018 - absoluto
cargo: Deputado Estadual - ano: 2018 - relativo


In [3]:
#hide
#collapse_output

## Vereador - 2020

ano = 2020
cargo = 'VEREADOR'
# cargo = 'Vereador'
municipio= 'RIO DE JANEIRO'
selection_dict = {'NR_TURNO':n_turno,'NM_MUNICIPIO':municipio,'DS_CARGO':cargo}
    

print(f'selection_dict: {selection_dict}')
group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
selected_col='NR_VOTAVEL'
nr_partidos=[13,50,65]
relative = False
only_elected=True


dict_candidatxs = get_dict_candidatxs(uf,ano,cargo,partidos,only_elected,epp,nr_partidos)
# dict_candidatxs
            
df_bairros = lab.get_group_results(ano,uf,selection_dict,group_level,selected_col,relative,dict_candidatxs)
df_bairros     

df_style = get_df_style(df_bairros,epp,ano,uf,n_turno,cargo,municipio,partidos,only_elected,group_level,selected_col,relative, nr_partidos,nrows)
df_style
# df_style.background_gradient()

selection_dict: {'NR_TURNO': 1, 'NM_MUNICIPIO': 'RIO DE JANEIRO', 'DS_CARGO': 'VEREADOR'}
Adicionando resultado...
Lendo dados do arquivo 'data/votacao_secao_2020_RJ/votacao_secao_2020_RJ.csv'.
Adicionando resultado...
Lendo dados do arquivo 'data/votacao_secao_2020_RJ/votacao_secao_2020_RJ.csv'.
Adicionando locais de votação...
Lendo dados do arquivo 'data/eleitorado_local_votacao_2020/eleitorado_local_votacao_2020.csv'.
Atenção! Existem linhas distintas com o mesmo id. df.id.nunique() = 596441, df.shape[0] = 597429


,NM_BAIRRO,13_Partido dos Trabalhadores,65_Partido Comunista do Brasil,50_Partido Socialismo e Liberdade,13000_NATHÁLIA MARTINS DE ASSIS,13007_ANTONIO ALDENISO DE OLIVEIRA,13010_JULIO CESAR SILVA DA COSTA,13021_ELIKA TAKIMOTO,13321_JOÃO RAPHAEL RAMOS DOS SANTOS,65065_JOAO LOURENCO XAVIER,65123_MARCIO AYER CORREIA ANDRADE,50123_TARCISIO MOTTA DE CARVALHO,50444_LUIZ FERNANDO DA CONCEICAO PALMARES,50500_FRANCISCO RODRIGUES DE ALENCAR FILHO,50555_RENATO ATHAYDE SILVA,13123_LUIZ LINDBERGH FARIAS FILHO,65180_BARBARA BAHIA HOLANDA MELO,50180_LUCIANA BOITEUX DE FIGUEIREDO RODRIGUES,50321_ARLINDO SARAIVA PEREIRA,50040_KAWANY PEDROSA LOPES TOME,50999_HELOISA ETERNA GONZAGA,65432_ANTONIO CLAUDIO SANTOS MENEZES,50234_SERGIO RICARDO DE LIMA,50888_ARILDO NERYS DA SILVA JUNIOR,65021_EDSON BAPTISTA DE SANTANA,65555_MARCELO QUEIROZ DOS SANTOS,50000_LEONEL BRIZOLA,50050_JOAO PAULO MARQUES ALVARO,50005_FELIPE CANDIDO GUEDES,13100_SÓLON CUNHA RUBEM,65651_NEY SÉRGIO COELHO FILHO,65657_PEDRO AMÉRICO MARTINS DE ALMEIDA,65777_CARLOS MONTEIRO,13333_REIMONT LUIZ OTONI SANTA BARBARA,50333_MONICA TEREZA AZEREDO BENICIO,13222_EDSON SANTOS DE SOUZA,50515_JULIANA SCHMITZ,13133_EDUARDO COELHO DE LIMA,13567_RICARDO DA SILVA SOUZA,65000_RAFAEL SANTIAGO FLORIANO,13777_TAINÁ REIS DE PAULA KAPAZ,50021_PEDRO HENRIQUE DE SOUSA TAVARES,50100_JOAO BATISTA OLIVEIRA DE ARAUJO,13131_GABRIEL APARECIDO DE OLIVEIRA,65222_FELIPE DE SOUZA RIBEIRO NUNES,65656_GLAUBER JOSÉ DE OLIVEIRA AMANCIO,65444_EDSON PAIVA DE OLIVEIRA,50111_PAULO PINHEIRO,50222_WILLIAM CARLOS BRUM BISPO,65020_GELSON DA COSTA,13001_MONICA MARTINS RABELO,13013_JOÃO PAULO MARTINS FARELLI,65658_LEDA MARA DE ARAÚJO,50200_BRUNO RAFAEL DE OLIVEIRA,13111_DOUGLAS SOARES SILVA,65655_ELEN OLIVEIRA FERREIRA SILVA,65007_ISAAC LOUREIRO JÚNIOR,13888_ANDRÉ DA SILVA,65333_ELIANE STELLET ALVES COELHO,13248_ANITA RODRIGUES TEIXEIRA,65955_CLARA MARIA FONSECA SILVA,65005_THAYLLA REGINA FRAZÃO DE ASSUMPÇÃO,65100_ALCIMAR PEREIRA BATISTA,65001_CHARLE AMERICANO DO BRASIL,65565_FLÁVIO DE CARVALHO BORGES,65006_ALESSANDRO CARMO DE ASSUMPCAO RABELLO,13604_ADILSON LUZIARIA,13444_KAROLINE NASCIMENTO MIRANDA,65888_MARCIO LOPES DIAS


In [ ]:
#hide
def reconstruct_html(fname):
    geo_json_file='../assets/html/maps/geo_json_coords.txt'

    with open(geo_json_file,'r') as f:
        geo_json_str = f.read()
    with open(fname,'r') as f:
        lines = f.readlines()
    out_str = ''
    for i,line in enumerate(lines):
        if '###geo_json###' in line:
            line = line.replace('###geo_json###','').rstrip() + geo_json_str

        out_str += line

    fname = fname.rstrip('.html') + '_reconst.html'
    with open(fname,'w') as f:
        f.write(out_str)
    print(len(out_str))
# reconstruct_html(fname)